# Optiver Realized Volatility Prediction : EDA + Data Preprocessing + Parameter Tuning + Feature Engineering + Modelling

#  **Importing the Libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
from sklearn.metrics import r2_score
import glob

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing
from sklearn import model_selection




import warnings
warnings.filterwarnings('ignore')
from joblib import Parallel, delayed
import scipy as sc
from sklearn.model_selection import train_test_split, KFold
import lightgbm as lgb
from lightgbm import LGBMRegressor
pd.set_option('max_columns', 300)

In [ ]:
train = pd.read_csv("../input/optiver-realized-volatility-prediction/train.csv")
test = pd.read_csv("../input/optiver-realized-volatility-prediction/test.csv")
train.head()

1. Now here we have to predict the target coloumn which is the volality of the optiver firm and the stocks they operate on. We will look at the distribution of the target column now.

In [ ]:
test.head()

## Target Distribution:
Target is skewed on left side

In [ ]:
mean = np.mean(train['target'])
print(f"Mean : {mean}")

plt.figure(figsize=(8, 5))
sns.distplot(train['target'], bins=50)
plt.title('Target Distribution')
plt.show()

Looking at the graph there are some outliers beyond a certain point. We will have look at them.

In [ ]:
print(f"Target count greater than 0.02 : {train['target'][train['target'] >= 0.02].count()}")
print(f"Percentage of total: {(train['target'][train['target'] >= 0.02].count() / train.shape[0]) * 100} %")

1. there's overall 0.3% of the target values which are greater than the 0.02. This means they are more volatile and are in demand. Now lets look at the mean distribution of the target.

In [ ]:
stock = train.groupby('stock_id')['target'].agg(['mean', 'sum']).reset_index()
print(f"Mean: {stock['mean'].mean()}")
print(f"Max value: {stock['sum'].mean()}")
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
sns.histplot(x=stock['mean'], ax=ax1)
sns.histplot(x=stock['sum'], ax=ax2)
ax1.set_title('Target mean distribution', size=12)
ax2.set_title('Target sum distribution', size=12)
plt.legend()
plt.show()

<b> So the mean value 0.003 which is close to 0 and Max value is on 14.8.</b>

In [ ]:
book_train = pd.read_parquet("../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0")
book_test = pd.read_parquet("../input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0")

trade_train = pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0")
trade_test = pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0")

book_train.head()

In [ ]:
df_book = book_train[book_train['time_id'] == 5]
df_book.head()

In [ ]:
plt.figure(figsize=(15, 5))
for col in ['bid_price1', 'bid_price2', 'ask_price1', 'ask_price2']:
    sns.lineplot(x='seconds_in_bucket', y=col, data=df_book, label=col)
plt.legend()
plt.show()

In [ ]:
df_trade= trade_train[trade_train['time_id'] == 5]
df_trade.head()

In [ ]:
plt.figure(figsize=(15, 5))
for col in ['bid_price1', 'bid_price2', 'ask_price1', 'ask_price2']:
    sns.lineplot(x='seconds_in_bucket', y=col, data=df_book, label=col)
    
sns.lineplot(x='seconds_in_bucket', y='price', data=df_trade, linewidth=3, color='black', label='price', )
plt.legend()
plt.show()

Now lets look at the most volatile time buckets in the data:

In [ ]:
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)

fig, ax = plt.subplots(figsize=(32, 10))
ax.barh(
    y=np.arange(10),
    width=train.sort_values(by='target', ascending=True).head(10)['target'],
    align='center',
    ecolor='black',
)

ax.set_yticks(np.arange(10))
ax.set_yticklabels(train.sort_values(by='target', ascending=True).head(10)['row_id'])
ax.set_xlabel('target', size=20, labelpad=15)
ax.set_ylabel('row_id', size=20, labelpad=15)
ax.tick_params(axis='x', labelsize=20, pad=10)
ax.tick_params(axis='y', labelsize=20, pad=10)
ax.set_title('Top 10 Least Volatile Time Buckets', size=25, pad=20)

plt.show()



# **Data Preprocessing**

In [ ]:
df_book['wap'] = (df_book['bid_price1'] * df_book['ask_size1']+df_book['ask_price1'] * df_book['bid_size1'])  / (df_book['bid_size1'] + df_book['ask_size1'])

def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

df_book.loc[:,'log_return'] = log_return(df_book['wap'])
df_book = df_book[~df_book['log_return'].isnull()]

In [ ]:
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))
realized_vol = realized_volatility(df_book['log_return'])
print(f'Realized volatility for stock_id 0 on time_id 5 is {realized_vol}')

In [ ]:
list_order_book_file_train = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet/*')

**Regression Model**

In [ ]:
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)

model_dict = {}

def realized_volatility_per_time_id_linear(file_path, prediction_column_name, train_test = True):
    df_book_data = pd.read_parquet(file_path)
    df_book_data['wap'] =(df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])  / (
                                      df_book_data['bid_size1']+ df_book_data[
                                  'ask_size1'])
    df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
    df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
    df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index()
    df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':prediction_column_name})
    stock_id = file_path.split('=')[1]
    df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')
    
    poly = PolynomialFeatures(degree=3)
    
    if train_test:
        
        df_realized_vol_per_stock_joined = train.merge(df_realized_vol_per_stock[['row_id',prediction_column_name]], on = ['row_id'], how = 'right')

        weights = 1/np.square(df_realized_vol_per_stock_joined.target)

        X = np.array(df_realized_vol_per_stock_joined[[prediction_column_name]]).reshape(-1, 1)
        X_ = poly.fit_transform(X)
        y = df_realized_vol_per_stock_joined.target


        reg = LinearRegression().fit(X_, y, sample_weight = weights)
        df_realized_vol_per_stock[[prediction_column_name]] = reg.predict(X_)

        model_dict[stock_id] = reg

    else: 
        
        reg = model_dict[stock_id]
        
        X = np.array(df_realized_vol_per_stock[[prediction_column_name]]).reshape(-1, 1)
        X_ = poly.fit_transform(X)
        df_realized_vol_per_stock[[prediction_column_name]] = reg.predict(X_)
    
    return df_realized_vol_per_stock[['row_id',prediction_column_name]]

In [ ]:
test.shape

In [ ]:
def past_realized_volatility_per_stock_linear(list_file,prediction_column_name, train_test = True):
    df_past_realized = pd.DataFrame()
    for file in list_file:
        df_past_realized = pd.concat([df_past_realized,
                                     realized_volatility_per_time_id_linear(file,prediction_column_name,train_test)])
    return df_past_realized

df_past_realized_train = past_realized_volatility_per_stock_linear(list_file=list_order_book_file_train,prediction_column_name='pred')

In [ ]:
train = train[['row_id','target']]
df_joined = train.merge(df_past_realized_train[['row_id','pred']], on = ['row_id'], how = 'left')

In [ ]:
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))
RMSPE = round(rmspe(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
print(f'Accuracy of the Regression model: RMSPE: {RMSPE}')

We will look at the difference between log return and the weighted average price

In [ ]:
plt.figure(figsize=(12,8)) 
plt.plot(book_train['seconds_in_bucket'],book_train['wap1'],label='WAP1')
plt.plot(book_train['seconds_in_bucket'],book_train['wap2'],label='WAP2')
plt.xlabel("seconds_in_bucket")
plt.ylabel("WAP2")
plt.legend()
plt.title("WAP1 and WAP2 of stock0 at time_id_5 w.r.t seconds in buckets")

**Polynomial regression**

Now let us look at the polynomial regression, first we will need to recover the original data frames;

In [ ]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

def calculate_wap(df):
    a1 = df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']
    b1 = df['bid_size1'] + df['ask_size1']
    a2 = df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']
    b2 = df['bid_size2'] + df['ask_size2']
    
    x = (a1/b1 + a2/b2)/ 2
    
    return x
def calculate_wap2(df):
        
    a1 = df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']
    a2 = df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']
    b = df['bid_size1'] + df['ask_size1'] + df['bid_size2']+ df['ask_size2']
    
    x = (a1 + a2)/ b
    return x

def calculate_wap3(df):
    a1 = df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']
    b1 = df['bid_size1'] + df['ask_size1']
    x = a1/b1
    return x

def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))



In [ ]:
def get_stock_stat(stock_id : int, dataType = 'train'):
    
    book_train_subset = pd.read_parquet(f'../input/optiver-realized-volatility-prediction/book_{dataType}.parquet/stock_id={stock_id}/')
    book_train_subset.sort_values(by=['time_id', 'seconds_in_bucket'])

    book_train_subset['bas'] = (book_train_subset[['ask_price1', 'ask_price2']].min(axis = 1)
                                / book_train_subset[['bid_price1', 'bid_price2']].max(axis = 1)
                                - 1)                               

    book_train_subset['wap'] = calculate_wap(book_train_subset)

    book_train_subset['log_return'] = (book_train_subset.groupby(by = ['time_id'])['wap'].
                                       apply(log_return).
                                       reset_index(drop = True).
                                       fillna(0)
                                      )
    
    stock_stat = pd.merge(
        book_train_subset.groupby(by = ['time_id'])['log_return'].agg(realized_volatility).reset_index(),
        book_train_subset.groupby(by = ['time_id'], as_index = False)['bas'].mean(),
        on = ['time_id'],
        how = 'left'
    )
    
    stock_stat.insert(0, "stock_id", stock_id)  
    
    return stock_stat

In [ ]:
def get_dataSet(stock_ids : list, dataType = 'train'):

    stock_stat = Parallel(n_jobs=-1)(
        delayed(get_stock_stat)(stock_id, dataType) 
        for stock_id in stock_ids
    )
    
    stock_stat_df = pd.concat(stock_stat, ignore_index = True)

    return stock_stat_df

In [ ]:
train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train.head()

In [ ]:
train_dataset = pd.read_csv("../input/processed-data/optiver-realized-volatility-datasets.csv")
train_dataset.head()

In [ ]:
df = train_dataset[['target','log_return','bas']]
df

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(20,10))
x_data, y_data = (df["log_return"].values, df["target"])
plt.plot(x_data, y_data, 'ro')
plt.ylabel('target')
plt.xlabel('WAP')
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
x_data, y_data = (df["log_return"], df["target"])

plt.plot(x_data - y_data, 'ro')
plt.ylabel('target')
plt.xlabel('log_return')
plt.show()

In [ ]:
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
#train_x = np.asanyarray(train[['log_return','bas']])
#train_y = np.asanyarray(train[['target']])
train_x = train[['log_return','bas']]
train_y = train['target']

test_x = test[['log_return','bas']]
test_y = test['target']

poly = PolynomialFeatures(degree=3)
train_x_poly = poly.fit_transform(train_x)

In [ ]:
weights = 1/np.square(train.target)

clf = linear_model.LinearRegression()
train_y_ = clf.fit(train_x_poly, train_y, sample_weight = weights)
# The coefficients
#print ('Coefficients: ', clf.coef_)
#print ('Intercept: ',clf.intercept_)

In [ ]:
test_x_poly = poly.fit_transform(test_x)


In [ ]:
from sklearn.metrics import r2_score

test_x_poly = poly.fit_transform(test_x)
test_y_ = clf.predict(test_x_poly)

#RMSPE = round(rmspe(y_true = test_y_['target'], test_x = df_joined['pred']),3)
#print("RMSPE Value for Polynomial Regression(Degree 3): %f", RMSPE)

# **Parameter Tuning and Light GBM**

Since we are using the same notebook we will need to import the data again and preprocess it.

In [ ]:
# Dataset path
data_path = Path('../input/optiver-realized-volatility-prediction')
# setting display option
pd.options.display.max_columns = 50

In [ ]:
# Objective variable
target = 'target'

# submission file setting
submit_file = 'submission.csv'
Id_column = 'row_id'

In [ ]:
#log returns
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

# Realized Volatility
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))


# WAP calculation
def wap_calculation1(df):
    return (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])

def wap_calculation2(df):
    return (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])

In [ ]:
# RMSPE
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

**Preprocessing the Book data**

In [ ]:
def book_preprocessing(stock_id : int, data_type = 'train'):
    # read data
    df = pd.read_parquet(data_path / f'book_{data_type}.parquet/stock_id={stock_id}/')
    
    # set stock_id
    df['stock_id'] = stock_id
    
    # WAP calculation
    df['wap1'] = wap_calculation1(df)
    df['wap2'] = wap_calculation2(df)
    
    # log return calculation
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return).fillna(0)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return).fillna(0)    
    # Log_return calculation each stock_id and time_id
    df_realized_vol_per_stock = pd.DataFrame(df.groupby(['stock_id','time_id'])[['log_return1','log_return2']].agg(realized_volatility)).reset_index()
    
    return df_realized_vol_per_stock


Like we have done above we calculate the log returns for the boo data by using the following function

In [ ]:
df_book = book_preprocessing(97, 'train')
df_book.head()

In [ ]:
def trade_preprocessing(stock_id : int, data_type = 'train'):
    # read data
    df = pd.read_parquet(data_path / f'trade_{data_type}.parquet/stock_id={stock_id}/')
    
    df = df.sort_values(by=['time_id', 'seconds_in_bucket']).reset_index(drop=True)
    
    # set stock_id
    df['stock_id'] = stock_id
    
    # log return calculation
    df['trade_log_return1'] = df.groupby(by = ['time_id'])['price'].apply(log_return).fillna(0)
    
    # Log_return calculation each stock_id and time_id
    df = pd.DataFrame(df.groupby(['stock_id','time_id'])[['trade_log_return1']].agg(realized_volatility).reset_index())
    
    return df

As done similarly above we use the following function to preprocess for the trade data 

In [ ]:
df_trade = trade_preprocessing(0,'train')
df_trade.head()

**Merging the book and train data**

In [ ]:
def get_stock_stat(stock_id : int, data_type = 'train'):
    
    # parquet data processing
    book_stat = book_preprocessing(stock_id, data_type)
    trade_stat = trade_preprocessing(stock_id, data_type)
    
    #Merge book and trade features
    stock_stat = book_stat.merge(trade_stat, on=['stock_id', 'time_id'], how='left').fillna(-999)
    
    return stock_stat

In [ ]:
def get_dataSet(stock_ids : list, data_type = 'train'):
    # Parallel process of get_stock_stat 
    stock_stat = Parallel(n_jobs=-1)(
        delayed(get_stock_stat)(stock_id, data_type) 
        for stock_id in stock_ids
    )
    # concat several stock_stats in vertical direction, axis=0(default)
    stock_stat_df = pd.concat(stock_stat, ignore_index = True)

    return stock_stat_df

In [ ]:
train=pd.read_csv(data_path / 'train.csv')
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
display(train.head())
print('train data shape:', train.shape)

In [ ]:
train_stock_stat_df = get_dataSet(stock_ids = train['stock_id'].unique(), data_type = 'train')

# Merge train with train_stock_stat_df
train = pd.merge(train, train_stock_stat_df, on = ['stock_id', 'time_id'], how = 'left')
print(f'Train shape: {train.shape}')
display(train.head(5))

In [ ]:
test = pd.read_csv(data_path /'test.csv')
test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
display(test.head())
print('test data shape:', test.shape)

In [ ]:
test_stock_stat_df = get_dataSet(stock_ids = test['stock_id'].unique(), data_type = 'test')
test = pd.merge(test, test_stock_stat_df, on = ['stock_id', 'time_id'], how = 'left').fillna(0)
print(f'Test shape: {test.shape}')
display(test.head(5))

In [ ]:
# Parameters of Light GBM
params_lgbm = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'learning_rate': 0.01,
        'objective': 'regression',
        'metric': 'None',
        'max_depth': -1,
        'n_jobs': -1,
        'feature_fraction': 0.7,
        'bagging_fraction': 0.7,
        'lambda_l2': 1,
        'verbose': -1
        #'bagging_freq': 5
}

In [ ]:
# Define loss function for lightGBM training
def feval_RMSPE(preds, train_data):
    labels = train_data.get_label()
    return 'RMSPE', round(rmspe(y_true = labels, y_pred = preds),5), False

In [ ]:
# training function
def light_gbm(X_train, y_train, X_val ,y_val, cats):
    
    # Create dataset
    train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cats, weight=1/np.power(y_train,2))
    val_data = lgb.Dataset(X_val, label=y_val, categorical_feature=cats, weight=1/np.power(y_val,2))
    
    # training
    model = lgb.train(params_lgbm, 
                      train_data, 
                      n_rounds, 
                      valid_sets=val_data, 
                      feval=feval_RMSPE,
                      verbose_eval= 250,
                      early_stopping_rounds=500
                     )
    
    # Prediction w/ validation data
    # preds_val = model.predict(train.loc[val_index, features_columns])
    preds_val = model.predict(X_val)

    # RMSPE calculation
    score = round(rmspe(y_true = y_val, y_pred = preds_val),5)

    # Prediction w/ validation data
    test_preds = model.predict(test[features_columns]).clip(0,1e10)
    
    # delete dataset
    del train_data, val_data
    
    return score, model

In [ ]:
# Categorical data column list
cats = ['stock_id']

model_name = 'lgb1'
pred_name = f'pred_{model_name}'

features_columns = ['stock_id', 'log_return1', 'log_return2', 'trade_log_return1']
print(f'Train dataset columns : {len(features_columns)} features')

train[pred_name] = 0
test[target] = 0

# k-flods Ensemble Training
n_folds = 4
n_rounds = 10000

kf = model_selection.KFold(n_splits=n_folds, shuffle=True, random_state=42)

# Initialize scores dict
scores_folds = {}
# Initialize value in scores_folds(dict) to record each step in CV
scores_folds[model_name] = []

# Initial value
cv_trial = 1

# --- Cross Validation ---
for train_index, val_index in kf.split(range(len(train))):
    
    print(f'CV trial : {cv_trial} /{n_folds}')
    
    # Divide dataset into train and validation data such as Cross Validation
    X_train = train.loc[train_index, features_columns]
    y_train = train.loc[train_index, target].values
    X_val = train.loc[val_index, features_columns]
    y_val = train.loc[val_index, target].values
    
    # train with Light GBM
    rmspe_score, model = light_gbm(X_train, y_train, X_val ,y_val, cats)
    
    # record score data at each train in CV
    scores_folds[model_name].append(rmspe_score)

    # Each validation Summary 
    print(f'Fold-{cv_trial} Model-{model_name} RMSPE: {rmspe_score}')
    print('-'*50)
    
    # Prediction w/ validation data
    test_preds = model.predict(test[features_columns]).clip(0,1e10)

    test[target] += test_preds
    cv_trial += 1

In [ ]:
# devide test target score into n_folds due to sum 4 preds value in CV process
test[target] = test[target]/n_folds

# score calculation
score = round(rmspe(y_true = train[target].values, y_pred = train[pred_name].values),5)
print(f'RMSPE {model_name}: {score} - Folds: {scores_folds[model_name]}')

display(test[[Id_column, target]].head(2))

In [ ]:
test[[Id_column, target]].to_csv(submit_file, index = False)